<a href="https://colab.research.google.com/github/thomasshin/NLP_Study/blob/main/Andrej_Karpathy_Neural_Network_Zero_to_Hero/Exercises_from_lecture_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/makemore-master"

LICENSE  makemore.py  names.txt  README.md


In [ ]:
!cp "/content/drive/My Drive/makemore-master/names.txt" "names.txt"

In [ ]:
!ls

drive  names.txt  sample_data


In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
chars = list(sorted(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

### **E01**
train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [ ]:
#create the training set of trigrams (x, y)
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append([ix1,ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
xs.shape,ys

(torch.Size([196113, 2]), tensor([13, 13,  1,  ..., 26, 24,  0]))

In [ ]:
import torch.nn.functional as F
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2,27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for i in range(100):
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the neural network: one-hot encoding (has to be float)
  xenc_flat = xenc.view(196113,-1)
  logits = xenc_flat @ W # predict log-counts
  counts = logits.exp() #counts, equivalent to N
  probs = counts / counts.sum(dim = 1, keepdims = True) # probabilities for the next character
  num = xenc_flat.size()[0]
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean() # model smoothing

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -20 * W.grad
print(loss.item())

2.3165528774261475


loss of trigram model with 100 epochs: 2.3165528774261475

loss of bigram model with 100 epochs: 2.522418260574341

so trigram model has improved over bigram model

In [ ]:
# sample from the neural net model
g = torch.Generator().manual_seed(2147483647)

for k in range(5):

  out = []
  ix0 = 0
  ix1 = 0
  while True:

    #---------
    # Before:
    # p = p[ix]
    #---------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix0, ix1]), num_classes=27).float()
    xenc_flat = xenc.view(-1,54)
    logits = xenc_flat @ W
    counts = logits.exp()
    p = counts / counts.sum(dim=1, keepdims=True)
    #---------

    ix0 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix0])
    if ix0 == 0:
      break
  print(''.join(out))

npo.
auw.
mnhjynrozkgq.
drcjqadraa.
anakxmrzapoa.


### **E02**
split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [ ]:
import random
random.seed(1)
random.shuffle(words)

In [ ]:
#bigram
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
X_train, y_train = xs[: int(len(xs)*0.8)], ys[: int(len(xs)*0.8)]
X_dev, y_dev = xs[int(len(xs)*0.8) : int(len(xs)*0.9)], ys[int(len(xs)*0.8) : int(len(xs)*0.9)]
X_test, y_test = xs[int(len(xs)*0.9) :], ys[int(len(xs)*0.9) :]

In [ ]:
g = torch.Generator().manual_seed(2147483647)
W_b = torch.randn((27,27), generator = g, requires_grad = True)

In [ ]:
# gradient descent
for i in range(100):
  # forward pass
  xenc = F.one_hot(X_train, num_classes=27).float()
  logits = xenc @ W_b
  counts = logits.exp()
  probs = counts / counts.sum(dim=1, keepdims=True)
  num = xenc.shape[0]
  loss = -probs[torch.arange(num), y_train].log().mean() + 0.01*(W_b**2).mean() # model smoothing with regularization

  # backward pass
  W_b.grad = None # set to zero the gradient
  loss.backward()

  # update
  W_b.data += -20 * W_b.grad
print("train loss for bigram :",loss.item())

train loss for bigram : 2.522306442260742


In [ ]:
xdevenc = F.one_hot(X_dev, num_classes=27).float()
logits = xdevenc @ W_b
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xdevenc.shape[0]
loss = -probs[torch.arange(num), y_dev].log().mean() + 0.01*(W_b**2).mean() # model smoothing with regularization
print("dev loss for bigram :", loss.item())

dev loss for bigram : 2.513911724090576


In [ ]:
xtestenc = F.one_hot(X_test, num_classes=27).float()
logits = xtestenc @ W_b
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.01*(W_b**2).mean() # model smoothing with regularization
print("test loss for bigram :",loss.item())

test loss for bigram : 2.5331950187683105


In [ ]:
#trigram
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append([ix1, ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
X_train, y_train = xs[: int(len(xs)*0.8)], ys[: int(len(xs)*0.8)]
X_dev, y_dev = xs[int(len(xs)*0.8) : int(len(xs)*0.9)], ys[int(len(xs)*0.8) : int(len(xs)*0.9)]
X_test, y_test = xs[int(len(xs)*0.9) :], ys[int(len(xs)*0.9) :]

In [ ]:
g = torch.Generator().manual_seed(2147483647)
W_t = torch.randn((27*2,27), generator = g, requires_grad = True)

In [ ]:
# gradient descent
for i in range(100):
  # forward pass
  xenc = F.one_hot(X_train, num_classes=27).float()
  xenc_flat = xenc.view(-1, 54)
  logits = xenc_flat @ W_t
  counts = logits.exp()
  probs = counts / counts.sum(dim=1, keepdims=True)
  num = xenc_flat.shape[0]
  loss = -probs[torch.arange(num), y_train].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization

  # backward pass
  W_t.grad = None # set to zero the gradient
  loss.backward()

  # update
  W_t.data += -20 * W_t.grad
print("train loss for trigram :", loss.item())

train loss for trigram : 2.3158535957336426


In [ ]:
xdevenc = F.one_hot(X_dev, num_classes=27).float()
xdevenc_flat = xdevenc.view(-1, 54)
logits = xdevenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xdevenc.shape[0]
loss = -probs[torch.arange(num), y_dev].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization
print("dev loss for trigram :", loss.item())

xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = xtestenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization
print("test loss for trigram :",loss.item())

dev loss for trigram : 2.3124120235443115
test loss for trigram : 2.329163074493408


train/dev/test losses for trigram are all lower than bigram.

### **E03**
use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [ ]:
reg_param = torch.linspace(0.001, 1, 10)
xdevenc = F.one_hot(X_dev, num_classes=27).float()
xdevenc_flat = xdevenc.view(-1, 54)
logits = xdevenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xdevenc.shape[0]

for param in reg_param:
  loss = -probs[torch.arange(num), y_dev].log().mean() + param*(W_t**2).mean() # model smoothing with regularization
  print("dev loss for trigram (param : {:10.3f}):".format(param), loss.item()) # 0.001 works the best

dev loss for trigram (param :      0.001): 2.305593967437744
dev loss for trigram (param :      0.112): 2.3896830081939697
dev loss for trigram (param :      0.223): 2.4737720489501953
dev loss for trigram (param :      0.334): 2.557861089706421
dev loss for trigram (param :      0.445): 2.6419501304626465
dev loss for trigram (param :      0.556): 2.726039171218872
dev loss for trigram (param :      0.667): 2.8101282119750977
dev loss for trigram (param :      0.778): 2.8942172527313232
dev loss for trigram (param :      0.889): 2.978306293487549
dev loss for trigram (param :      1.000): 3.0623950958251953


In [ ]:
xenc = F.one_hot(X_train, num_classes=27).float()
xenc_flat = xenc.view(-1, 54)
logits = xenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xenc_flat.shape[0]

for param in reg_param:
  loss = -probs[torch.arange(num), y_train].log().mean() + param*(W_t**2).mean() # model smoothing with regularization
  print("train loss for trigram (param : {:10.3f}):".format(param), loss.item()) # 0.001 works the best

train loss for trigram (param :      0.001): 2.3082919120788574
train loss for trigram (param :      0.112): 2.392380952835083
train loss for trigram (param :      0.223): 2.4764699935913086
train loss for trigram (param :      0.334): 2.560559034347534
train loss for trigram (param :      0.445): 2.6446480751037598
train loss for trigram (param :      0.556): 2.7287371158599854
train loss for trigram (param :      0.667): 2.812826156616211
train loss for trigram (param :      0.778): 2.8969151973724365
train loss for trigram (param :      0.889): 2.981004238128662
train loss for trigram (param :      1.000): 3.0650930404663086


In [ ]:
#using reg_param of 0.001 for test loss
xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = xtestenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc_flat.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.001*(W_t**2).mean() # model smoothing with regularization
print("test loss for trigram :", loss.item()) # test loss has improved

test loss for trigram : 2.322345018386841


Tuning the strength of smoothing achieved test loss of 2.322345018386841, which is slightly lower than the original

test loss for original trigram : 2.329163074493408

### **E04**
we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [ ]:
W_0 = torch.randn((54,27), requires_grad = True, generator = g)

In [ ]:
for i in range(100):
  W_1, W_2 = W[:27], W[27:]
  logits = W_1[X_train[:, 0]] + W_2[X_train[:, 1]]
  counts = logits.exp()
  probs = counts / counts.sum(dim=1, keepdims=True)
  loss = -probs[torch.arange(X_train.shape[0]), y_train].log().mean() + 0.001*(W_0**2).mean()
  W_0.grad = None # set to zero the gradient
  loss.backward()
  lr = 0.01
  W_0.data += -lr * W_0.grad

print("train loss :", loss.item())

train loss : 2.309070110321045


In [ ]:
xdevenc = F.one_hot(X_dev, num_classes=27).float()
xdevenc_flat = xdevenc.view(-1, 54)
logits = W_1[X_dev[:, 0]] + W_2[X_dev[:, 1]]
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xdevenc_flat.shape[0]
loss = -probs[torch.arange(num), y_dev].log().mean() + 0.001*(W_0**2).mean() # model smoothing with regularization
print("dev loss :", loss.item())

dev loss : 2.301557779312134


In [ ]:
xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = W_1[X_test[:, 0]] + W_2[X_test[:, 1]]
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc_flat.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.001*(W_0**2).mean() # model smoothing with regularization
print("test loss :", loss.item())

test loss : 2.3189308643341064


Yes, we can use indexing to replace F.one_hot

### ***E05***
look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [ ]:
xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = xtestenc_flat @ W_t
#counts = logits.exp()
#probs = counts / counts.sum(dim=1, keepdims=True)
#num = xtestenc.shape[0]
#loss = -probs[torch.arange(num), y_test].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization
print("test loss for trigram :",F.cross_entropy(logits, y_test))

test loss for trigram : tensor(2.3216, grad_fn=<NllLossBackward0>)


In [ ]:
xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = xtestenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization
print("test loss for trigram :",loss.item())

test loss for trigram : 2.329163074493408


By using F.cross_entropy, we do not need to create counts, probs, num and loss variables, which minimize the memory use. Also, it makes computation more efficient when larger chunks of data are used

### ***E06***
meta-exercise! Think of a fun/interesting exercise and complete it.

In [ ]:
# increase epochs(trigram) to 1000 and update learning rate
for i in range(1000):
  # forward pass
  xenc = F.one_hot(X_train, num_classes=27).float()
  xenc_flat = xenc.view(-1, 54)
  logits = xenc_flat @ W_t
  counts = logits.exp()
  probs = counts / counts.sum(dim=1, keepdims=True)
  num = xenc_flat.shape[0]
  loss = -probs[torch.arange(num), y_train].log().mean() + 0.01*(W_t**2).mean() # model smoothing with regularization

  # backward pass
  W_t.grad = None # set to zero the gradient
  loss.backward()

  # update
  if i < 750:
    lr = 20
  else:
    lr = 10
  W_t.data += -lr * W_t.grad
print("train loss for trigram :", loss.item())

train loss for trigram : 2.2536697387695312


In [ ]:
xdevenc = F.one_hot(X_dev, num_classes=27).float()
xdevenc_flat = xdevenc.view(-1, 54)
logits = xdevenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xdevenc.shape[0]
loss = -probs[torch.arange(num), y_dev].log().mean() + 0.01*(W_t**2).mean()
print("dev loss for trigram :", loss.item())

dev loss for trigram : 2.252570629119873


In [ ]:
xtestenc = F.one_hot(X_test, num_classes=27).float()
xtestenc_flat = xtestenc.view(-1, 54)
logits = xtestenc_flat @ W_t
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdims=True)
num = xtestenc_flat.shape[0]
loss = -probs[torch.arange(num), y_test].log().mean() + 0.001*(W_t**2).mean()
print("test loss for trigram :", loss.item())

test loss for trigram : 2.2577171325683594
